# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score

# Load Data

In [ ]:
df = pd.read_csv("/mnt/hdd/Datasets/IOT-temp.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

# EDA

In [ ]:
df["noted_date"] = pd.to_datetime(df["noted_date"], format="%d-%m-%Y %H:%M")
df["year"] = df["noted_date"].apply(lambda date: date.year)
df["month"] = df["noted_date"].apply(lambda date: date.month)
df["day"] = df["noted_date"].apply(lambda date: date.day_name())
df["week_of_year"] = df["noted_date"].apply(lambda date: date.weekofyear)
df["hour"] = df["noted_date"].apply(lambda date: date.hour)
df["minute"] = df["noted_date"].apply(lambda date: date.minute)

In [ ]:
df.head()

In [ ]:
df["room_id/id"].value_counts()

In [ ]:
df.drop(["id", "room_id/id"], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
monthly_temp_mean = df.groupby(["out/in", "month"], as_index=False)[["temp"]].mean()
monthly_temp_mean

In [ ]:
plt.figure(figsize=(14, 5))
ax = sns.barplot(data=monthly_temp_mean, x="month", y="temp", hue="out/in")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Monthly Average Temperature")
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
sns.lineplot(data=monthly_temp_mean, x="month", y="temp", hue="out/in")
plt.title("Monthly Average Temperature")
plt.grid()
plt.show()

In [ ]:
weekly_temp_mean = df[df["month"] == 10].groupby(["week_of_year", "day", "out/in"], as_index=False)[["temp"]].mean()
weekly_temp_mean

In [ ]:
for week in weekly_temp_mean["week_of_year"].unique():
    weekly_temp = weekly_temp_mean[weekly_temp_mean["week_of_year"] == week]
    plt.figure(figsize=(14, 5))
    ax = sns.barplot(data=weekly_temp, x="day", y="temp", hue="out/in")
    for container in ax.containers:
        ax.bar_label(container)
    plt.title(f"Month: 10, Week of Year: {week} Average Temperature")
    plt.show()

    plt.figure(figsize=(14, 5))
    sns.kdeplot(data=weekly_temp, x="temp", hue="out/in", fill=True)
    plt.title(f"Month: 10, Week of Year: {week} Average Temperature")
    plt.show()

In [ ]:
plt.figure()
ax = sns.countplot(data=df, x="out/in")
ax.bar_label(ax.containers[0])
plt.title("Place Count")
plt.show()

In [ ]:
weekofyear_temp_mean = df.groupby(["week_of_year", "out/in"], as_index=False)[["temp"]].mean()
weekofyear_temp_mean

In [ ]:
plt.figure(figsize=(14, 5))
sns.lineplot(data=weekofyear_temp_mean, x="week_of_year", y="temp", hue="out/in")
plt.title("Week of Year Average Temperature")
plt.show()

# Preprocess

In [ ]:
df.head()

In [ ]:
in_temperatures = temp_df[temp_df["out/in"] == "In"][["temp"]]
out_temperatures = temp_df[temp_df["out/in"] == "Out"][["temp"]]

In [ ]:
in_temperatures = np.array(in_temperatures).reshape(-1, 1)
out_temperatures = np.array(out_temperatures).reshape(-1, 1)

In [ ]:
in_scaler = MinMaxScaler()
out_scaler = MinMaxScaler()

in_temperatures_scaled = in_scaler.fit_transform(in_temperatures)
out_temperatures_scaled = out_scaler.fit_transform(out_temperatures)

In [ ]:
in_train_size = int(len(in_temperatures_scaled) * 0.8)
out_train_size = int(len(out_temperatures_scaled) * 0.8)

In [ ]:
in_test_size = len(in_temperatures_scaled) - in_train_size
out_test_size = len(out_temperatures_scaled) - out_train_size

In [ ]:
print("In Train Size:", in_train_size)
print("In Test Size:", in_test_size)
print("Out Train Size:", out_train_size)
print("Out Test Size:", out_test_size)

In [ ]:
in_train = in_temperatures_scaled[0:in_train_size, :]
in_test = in_temperatures_scaled[in_train_size:len(in_temperatures_scaled), :]

In [ ]:
out_train = out_temperatures_scaled[0:out_train_size, :]
out_test = out_temperatures_scaled[out_train_size:len(out_temperatures_scaled), :]

In [ ]:
def dataset(data, steps=1):
    data_x, data_y = [], []
    for i in range(len(data) - steps - 1):
        a = data[i:(i + steps), 0]
        b = data[i + steps, 0]
        data_x.append(a)
        data_y.append(b)

    data_x = np.array(data_x)
    data_y = np.array(data_y)
    return data_x, data_y

In [ ]:
X_train_in, y_train_in = dataset(in_train)
X_test_in, y_test_in = dataset(in_test)

In [ ]:
X_train_out, y_train_out = dataset(out_train)
X_test_out, y_test_out = dataset(out_test)

# Model

In [ ]:
in_model = Sequential()
in_model.add(LSTM(16, input_shape=(1, 1)))
in_model.add(Dense(1))
in_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [ ]:
out_model = Sequential()
out_model.add(LSTM(64, input_shape=(1, 1)))
out_model.add(Dense(1))
out_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [ ]:
early = EarlyStopping(monitor="val_loss", patience=3)

In [ ]:
in_history = in_model.fit(X_train_in, y_train_in, epochs=100, validation_split=0.2, callbacks=[early])

In [ ]:
in_model.save("in_iot.h5")

In [ ]:
out_history = out_model.fit(X_train_out, y_train_out, epochs=100, validation_split=0.2, callbacks=[early])

In [ ]:
out_model.save("out_iot.h5")

# Predict

In [ ]:
y_pred_in = in_model.predict(X_test_in)
y_pred_in = in_scaler.inverse_transform(y_pred_in)

y_test_in = y_test_in.reshape(-1, 1)
y_test_in = in_scaler.inverse_transform(y_test_in)
print("R2 Score:", r2_score(y_test_in, y_pred_in))

In [ ]:
plt.figure(figsize=(13, 4))
plt.plot(y_test_in, label="Actual")
plt.plot(y_pred_in, label="Predicted")
plt.title("Predicted vs Actual (In)")
plt.legend()
plt.show()

In [ ]:
y_pred_out = out_model.predict(X_test_out)
y_pred_out = out_scaler.inverse_transform(y_pred_out)

y_test_out = y_test_out.reshape(-1, 1)
y_test_out = out_scaler.inverse_transform(y_test_out)
print("R2 Score:", r2_score(y_test_out, y_pred_out))

In [ ]:
plt.figure(figsize=(13, 4))
plt.plot(y_test_out, label="Actual")
plt.plot(y_pred_out, label="Predicted")
plt.title("Predicted vs Actual (Out)")
plt.legend()
plt.show()